In [6]:
from src.data_loader import GetJoeyData
from src.model import BabyJoeyModel
from src.training import ModelTrainer
from dataclasses import dataclass
import torch
import torch.nn.functional as F
from src.config import ModelConfig, TrainingConfig, DatasetConfig

dataset_instance = GetJoeyData(DatasetConfig())
training_dataset, validation_dataset = dataset_instance.create_dataloaders()

Loading raw datasets from disk at ./processed_data_Tweets_Australian_Cities...
Loading tokenized datasets from disk...


In [7]:
sample = next(iter(training_dataset))

In [8]:
sample


{'input_ids': tensor([[  361,   345,  4398,  ..., 50256, 50256, 50256],
         [   32,  6891, 13516,  ..., 50256, 50256, 50256]])}